In [4]:
# Importaciones
import numpy as np
import scipy as sci
import sympy as sp
import pandas as pd
import requests
import json
from math import *

<a href="https://colab.research.google.com/github/ComputationalMethods/Evaluacion_2022-1/blob/main/Examenes/Examen_2022_1_02_enunciado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examen 2: Métodos computacionales
A)
1. Usando su número de cédula cómo semilla, obtenga un numero entero aleatorio, $n$, entre 2 y 20.
2. Usando el $n$ obtenido, haga una estimación de la siguiente integral hasta dos cifras significativas, usando el método `quad` del módulo `integrate` de Scipy
$$
\int_0^\infty \sin(x^n)\operatorname{d}x
$$
Sugerencia. Compare con el valor obtenido desde https://www.wolframalpha.com/ o [sympy](https://stackoverflow.com/questions/65862611/integration-of-sin-x-dx-problem-in-python-programming-implementation)

In [5]:
# Punto 1 
np.random.seed(1214748203)
Valn = np.random.randint(2,20)
Valn

7

In [6]:
x = sp.symbols('x')
λ = Valn
inf = float("inf")
test = lambda x: sp.sin(x**λ)
def I(f):
    return sp.integrate(test(x),(x,0,inf))

In [7]:
print(I(test).subs(λ,7))

2**(1/7)*sqrt(pi)*gamma(4/7)/(14*gamma(13/14))


In [8]:
resultado = ( 2**(1/7)*sqrt(pi)*sci.special.gamma(4/7) ) / (14*sci.special.gamma(13/14))
resultado

0.20815444015266787

Valor obtenido en https://www.wolframalpha.com/ es: 

![](resultadoWA.PNG)

B) Cargue los datos disponibles en:

https://raw.githubusercontent.com/ComputationalMethods/Evaluacion_2022-1/main/data/lista_pares.json

como un DataFrame de pandas. Este contendrá dos columnas. La columna `lista`, con una lista de 8 enteros (algunos de ellos repetidos) y una columna `pares` correspondiente a un diccionario
con dos items: un entero rotulado con `'S'` y una lista de pares de enteros routulada con `'ψ'`. Note que el resultado de la suma en valor absoluto de los dos enteros en cada uno de los pares reproduce el valor del correspondiente `'S'` 

Seleccione la fila del DataFrame con el índice correspondiente a los dos últimos dígitos de su cédula y desarrolle los siguientes puntos con la correspondiente  `lista` y `pares`:
1. Construya una matrix $8\times 8$ _simétrica_ donde cada fila y la correspondiente columna correspondan a un entero de la `lista` (incluyendo los repetidos); de manera que en los cruces de filas y columnas definidos por los pares de enteros de `'ψ'`, se genere una entrada aleatoria mayor que cero y menor o igual que 1, con las demas entradas iguales a cero.
1. Calcule el determinante y establezca explícitamente si es diferente cero (tenga en cuenta la precisión numérica). Recuerde que si el determinante es cero, se espera que al menos uno de los autovalores sea cero
2. Organize los autovalores y autovectores de menor a mayor en valor absoluto y compruebe que las matriz de autovectores organizada da lugar a los autovalores en el orden correcto. 
3. Establezca explícitamente _cuantos_ autovalores son degenerados en valor absoluto.

In [9]:
Pagina = "https://raw.githubusercontent.com/ComputationalMethods/Evaluacion_2022-1/main/data/lista_pares.json"
Archivo = json.loads(requests.get(Pagina).text)
Df = pd.DataFrame(Archivo)
Df

,lista,pares
0,"[1, 1, 2, 3, -4, -4, -5, 6]","{'S': 2, 'ψ': [[3, -5], [2, -4], [-4, 6], [1, ..."
1,"[1, 2, 2, 2, -3, -5, -6, 7]","{'S': 4, 'ψ': [[2, 2], [-3, 7], [1, -5], [2, -..."
2,"[1, -2, -2, 4, 5, -7, -7, 8]","{'S': 9, 'ψ': [[4, 5], [1, 8], [-2, -7], [5, 4..."
3,"[1, 2, 2, 4, -5, -5, -7, 8]","{'S': 3, 'ψ': [[4, -7], [-5, 8], [2, -5], [1, ..."
4,"[2, -3, -4, 5, -6, 7, 7, -8]","{'S': 1, 'ψ': [[-6, 7], [7, -8], [5, -6], [2, ..."
...,...,...
240,"[2, -4, -5, 10, 11, -16, -17, 19]","{'S': 21, 'ψ': [[-4, -17], [-5, -16], [10, 11]..."
241,"[5, -7, -7, 11, 13, -17, -17, 19]","{'S': 24, 'ψ': [[5, 19], [-7, -17], [11, 13], ..."
242,"[4, -5, -7, 8, -17, 18, 18, -19]","{'S': 1, 'ψ': [[-7, 8], [4, -5], [-17, 18], [1..."
243,"[2, 2, 5, -7, 10, -14, -17, 19]","{'S': 12, 'ψ': [[2, -14], [2, 10], [-7, 19], [..."


In [33]:
MisDatos = Df.iloc[3,:]
Lista = MisDatos["lista"]
Pares = MisDatos["pares"]
print(Lista)
print(Pares)

[1, 2, 2, 4, -5, -5, -7, 8]
{'S': 3, 'ψ': [[4, -7], [-5, 8], [2, -5], [1, 2], [-7, 4], [8, -5], [-5, 2], [2, 1]]}


In [34]:
# Creo que la matriz que voy a llenar con los datos ramdom
MatrixSimetrica = np.zeros((8,8))
# Solo tomo los primeros valores correspondientes a la parte superior de la matriz
for i in range(0,int(len(Pares['ψ'])/2)):
    # pido la fila y la columna de cada valor
    fila = Lista.index(Pares['ψ'][i][0])
    columna = Lista.index(Pares['ψ'][i][1])
    # lo agrego a la columna
    MatrixSimetrica[fila,columna] = round(np.random.random(),3)
    # dado que hay valores repetidos, verifico y si coincide agrego un valor nuevo a la matriz
    if Lista.count(Pares['ψ'][i][1]) == 2:
        MatrixSimetrica[fila,columna+1] = round(np.random.random(),3)
# la vuelvo simetrica sumandole la transpuesta
MatrixSimetrica += MatrixSimetrica.transpose()
MatrixSimetrica

array([[0.   , 0.532, 0.5  , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.532, 0.   , 0.   , 0.   , 0.513, 0.74 , 0.   , 0.   ],
       [0.5  , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.247, 0.   ],
       [0.   , 0.513, 0.   , 0.   , 0.   , 0.   , 0.   , 0.004],
       [0.   , 0.74 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.247, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.004, 0.   , 0.   , 0.   ]])

In [35]:
determinante = np.linalg.det(MatrixSimetrica)
determinante

1.3363411359999997e-07

In [36]:
val, vect = np.linalg.eig(MatrixSimetrica)

In [40]:
val.round(3)

array([ 1.082, -1.082,  0.416, -0.416,  0.003, -0.003,  0.247, -0.247])

In [42]:
valAbs = abs(val).round(3)
valAbs

array([1.082, 1.082, 0.416, 0.416, 0.003, 0.003, 0.247, 0.247])

In [43]:
valOrdenado = sorted(valAbs)
valOrdenado

[0.003, 0.003, 0.247, 0.247, 0.416, 0.416, 1.082, 1.082]

In [50]:
vectRedon = vect.round(3)
vectRedon

array([[-0.425, -0.425, -0.565,  0.565, -0.   , -0.   , -0.   ,  0.   ],
       [-0.679,  0.679,  0.196,  0.196,  0.002, -0.002,  0.   ,  0.   ],
       [-0.196,  0.196, -0.679, -0.679, -0.002,  0.002, -0.   , -0.   ],
       [-0.   , -0.   ,  0.   , -0.   , -0.   , -0.   , -0.707, -0.707],
       [-0.322, -0.322,  0.242, -0.242, -0.581, -0.581,  0.   , -0.   ],
       [-0.465, -0.465,  0.349, -0.349,  0.403,  0.403,  0.   , -0.   ],
       [-0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.707,  0.707],
       [-0.001,  0.001,  0.002,  0.002, -0.707,  0.707, -0.   , -0.   ]])

In [48]:
# Creo la matriz que voy a llenar con los datos de los vectores
vectOrdenados = np.zeros((8,8))
# Dado que la matriz tiene lambda en valor absoluto duplicados, damos saltos de 2 en 2
for i in range(0,len(val),2):
    # Tomo los indices de los vectores que tengo que mover
    puntero1 = list(valAbs).index(valOrdenado[i],0,len(valAbs))
    puntero2 = list(valAbs).index(valOrdenado[i],puntero1+1,len(valAbs))
    # en el vector en la posición i e i+1 (lambdas dobles) agrego el vector de las respectivas posiciones como columna
    vectOrdenados = np.insert(vectOrdenados,i,vectRedon[:,puntero1],1)
    vectOrdenados = np.insert(vectOrdenados,i+1,vectRedon[:,puntero2],1)
    # elimino la parte del final ya que siempre me quedan ceros sobrando no los reemplaza solo inserta
    vectOrdenados = np.delete(vectOrdenados,len(vectOrdenados),1)
    vectOrdenados = np.delete(vectOrdenados,len(vectOrdenados),1)
vectOrdenados

array([[-0.   , -0.   , -0.   ,  0.   , -0.565,  0.565, -0.425, -0.425],
       [ 0.002, -0.002,  0.   ,  0.   ,  0.196,  0.196, -0.679,  0.679],
       [-0.002,  0.002, -0.   , -0.   , -0.679, -0.679, -0.196,  0.196],
       [-0.   , -0.   , -0.707, -0.707,  0.   , -0.   , -0.   , -0.   ],
       [-0.581, -0.581,  0.   , -0.   ,  0.242, -0.242, -0.322, -0.322],
       [ 0.403,  0.403,  0.   , -0.   ,  0.349, -0.349, -0.465, -0.465],
       [ 0.   ,  0.   , -0.707,  0.707,  0.   ,  0.   , -0.   ,  0.   ],
       [-0.707,  0.707, -0.   , -0.   ,  0.002,  0.002, -0.001,  0.001]])

Nota: desconozco el significado de autovalor degenerado en valor absoluto por ende no puedo responder esa pregunta